# LAB_auto.ipynb

Interactive pipeline for ankle torque analysis.

## Imports and Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.signal import resample_poly
import os
import re

# Configuration
root_dir = "/Users/sophialiu/Downloads/LAB/dephy"  # Replace with your actual root directory
output_dir = "output_torque"  # Output directory

# Create output directory
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Root directory: {root_dir}")
print(f"Output directory: {output_dir}")
print(f"Root directory exists: {os.path.exists(root_dir)}")

## File Discovery

In [ ]:
# Find all main TSV files (S0x_T0x.tsv format)
tsv_files = []

print("=== File Discovery Process ===")
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.tsv') and re.match(r'^S\d+_T\d+\.tsv$', file):
            print(f"\\nFound main TSV file: {file}")

            # Get file path
            position_path = os.path.join(root, file)

            # Parse main file information
            file_prefix = file[:-4]  # Remove .tsv

            # Extract trial number
            match = re.match(r'^S\\d+_T(\\d+)\\.tsv$', file)
            if match:
                trial_num = int(match.group(1))

                # Determine left/right foot force data files
                if trial_num in [5, 7]:
                    right_force_file = f"{file_prefix}_f_7.tsv"  # Right foot uses f_7
                    left_force_file = f"{file_prefix}_f_6.tsv"   # Left foot uses f_6
                    print(f"  Trial T{trial_num} - Left/right foot data swapped")
                else:
                    right_force_file = f"{file_prefix}_f_6.tsv"  # Right foot uses f_6
                    left_force_file = f"{file_prefix}_f_7.tsv"   # Left foot uses f_7
                    print(f"  Trial T{trial_num} - Standard left/right foot configuration")

                right_force_path = os.path.join(root, right_force_file)
                left_force_path = os.path.join(root, left_force_file)

                print(f"  Right foot force file: {right_force_file} - {'exists' if os.path.exists(right_force_path) else 'not found'}")
                print(f"  Left foot force file: {left_force_file} - {'exists' if os.path.exists(left_force_path) else 'not found'}")

                # Check if force data files exist and add to processing list
                if os.path.exists(right_force_path):
                    tsv_files.append((position_path, right_force_path, "RIGHT"))
                    print(f"  ✓ Added right foot data pair")

                if os.path.exists(left_force_path):
                    tsv_files.append((position_path, left_force_path, "LEFT"))
                    print(f"  ✓ Added left foot data pair")

print(f"\\n=== Summary ===")
print(f"Found {len(tsv_files)} matching position and force data file pairs")

# Display all matched file pairs
for i, (pos_path, force_path, side) in enumerate(tsv_files, 1):
    print(f"{i}. {os.path.basename(pos_path)} + {os.path.basename(force_path)} ({side})")

## Quick demo cell

In [ ]:
# ▶️ Demo: Run one file to see plots
if len(tsv_files) == 0:
    print("No TSV files discovered, please check root_dir.")
else:
    # pick first pair for demo
    demo_pos, demo_force, demo_side = tsv_files[0]
    # locate opposite foot force files for plate offsets
    right_fp = left_fp = None
    for p, f, s in tsv_files:
        if s == "RIGHT":
            right_fp = f
        elif s == "LEFT":
            left_fp = f
    demo_result = process_single_file(demo_pos, demo_force, demo_side,
                                      right_force_path=right_fp,
                                      left_force_path=left_fp)

    if demo_result:
        plot_ankle_position_interactive(demo_result)
        plot_torque_data_interactive(demo_result)